# Projet de Machine Learning

In [99]:
from tools import *
import math as math
import copy
%matplotlib inline

In [100]:
def gen_arti(centerx=1,centery=1,sigma=0.01,nbex=10,data_type=0,eps=0.01):
    """ Generateur de donnees,
        :param centerx: centre des gaussiennes
        :param centery:
        :param sigma: des gaussiennes
        :param nbex: nombre d'exemples
        :param data_type: 0: melange 2 gaussiennes, 1: melange 4 gaussiennes, 2:echequier
        :param eps: bruit dans les donnees
        :return: data matrice 2d des donnnes,y etiquette des donnnees
    """
    if data_type==0:
         #melange de 2 gaussiennes
         xpos=np.random.multivariate_normal([centerx,centery],np.diag([sigma,sigma]),nbex//2)
         xneg=np.random.multivariate_normal([-centerx,-centery],np.diag([sigma,sigma]),nbex//2)
         data=np.vstack((xpos,xneg))
         y=np.hstack((np.ones(nbex//2),-np.ones(nbex//2)))
    if data_type==1:
        #melange de 4 gaussiennes
        xpos=np.vstack((np.random.multivariate_normal([centerx,centery],np.diag([sigma,sigma]),nbex//4),
                        np.random.multivariate_normal([-centerx,-centery],np.diag([sigma,sigma]),nbex//4)))
        xneg=np.vstack((np.random.multivariate_normal([-centerx,centery],np.diag([sigma,sigma]),nbex//4),
                        np.random.multivariate_normal([centerx,-centery],np.diag([sigma,sigma]),nbex//4)))
        data=np.vstack((xpos,xneg))
        y=np.hstack((np.ones(nbex//2),-np.ones(nbex//2)))

    if data_type==2:
        #echiquier
        data=np.reshape(np.random.uniform(-4,4,2*nbex),(nbex,2))
        y=np.ceil(data[:,0])+np.ceil(data[:,1])
        y=2*(y % 2)-1
    # un peu de bruit
    data[:,0]+=np.random.normal(0,eps,nbex)
    data[:,1]+=np.random.normal(0,eps,nbex)
    # on mélange les données
    idx = np.random.permutation((range(y.size)))
    data=data[idx,:]
    y=y[idx]
    return data,y

    

In [101]:
def f_K(x, e, sigma=1):
    return math.exp(-pow(np.linalg.norm(x-e), 2)/(2*pow(sigma,2)))

# Calcul de tous les H(xl) pour le a minimal 
def HA(a, S, dataset, y):
    N = dataset.shape[0]
    Kql= np.zeros((N,N))
    f = np.zeros((0, 0))
    
    #print "Calcul des fl..."
    for l,x in enumerate(dataset):
        fl = 0
        for j,e in enumerate(S):
            fl += a[j]*f_K(x,e)
        f = np.append(f, fl)
        
    #print "Done"
    for i,x in enumerate(S):
        for j,e in enumerate(S):
            Kql[i,j] = f_K(x,e)
            
    Kql = np.asmatrix(np.asarray(Kql))
    f = np.asmatrix(f.reshape((f.shape[0], 1)))
    y = np.asmatrix(y)

    
    #print "Calcul de H(xl)..."
    return -sum(y.T*f - np.log(1+np.exp(f)))+lamda/2 * a.T * Kql * a

# Calcul le a minimal approximé par Newton-R
def H(x, S, a_prec, dataset, y):

    a = a_optim(a_prec) 
    return HA(a, S, dataset, y), a

def a_optim(a_prec):

    f = K*a_prec 
    print f
    p = np.multiply(np.asarray(np.exp(f)),1/np.asarray((1+np.exp(f))))
    d = np.asarray(p*(1-p))
    W = np.asmatrix(np.zeros((d.shape[0], d.shape[0])))
    
    for i,element in enumerate(d):
        W[i,i] = element
        
    z = K*a_prec+np.linalg.inv(W)*(y-p)
    return np.linalg.inv(K.T*W*K + lamda*K) * K.T*W*z 

def ivm(dataset, y):

    N = dataset.shape[0]
    a = np.zeros((N,1))
    R = np.zeros((N,dataset.shape[1]))
    S = []
    predict = None
    lamda = 0.1
    
    Hk = 1
    Hk_1 = 0
    for i, e in enumerate(dataset):
        R[i] = e

    while abs(Hk-Hk_1)/abs(Hk) > 0.01:
        if R.shape[0] == 0:
            break
            
        #print "Calcul des nouveaux H(xl)..."
        h = np.ones((R.shape[0], 1))
        S_b = []
        for e in S:
            S_b.append(e)
            
        A = []
        for i, xl in enumerate(R):
            S_b.append(xl)
            h[i], a = H(xl, S_b, a, dataset, y)
            A.append(a)
            S_b.pop()
            
            
        #print "Done"
        #print "On recupere le plus petit H(xl)"
        xl_opt = np.argmin(h)
        Hk_1 = Hk
        Hk = h[xl_opt]
        #print h
        #print "Le plus petit est ", Hk, " il est à l'indice : ", xl_opt

        #print "On l'ajoute a S..."

        S.append(R[xl_opt])
        #print "--------------> S:"

        #print "OK"
        #print "On l'enleve a R..."

        data = []
        for i, e in enumerate(R):
            if i == xl_opt:
                pass
            else:
                data.append(e)
        R = np.asarray(data)
        # update de a
        print "Updating a..."

        a = a_optim(A[xl_opt])
        #print "On garde celui associe au meilleur:", A[xl_opt]
        #print "OK"
        print "ecart", abs(Hk-Hk_1)/abs(Hk)
        
    print "S final"
    for el in S:
        print el
    print "Il y a ", len(S), " import points"
    
    print "a final"
    for e in a:
        print e
    return S, a

def predict(x, S, a):

    prediction = []
    for e in x:
        m = 0
        for i, s in enumerate(S):
            m += a[i]*f_K(e, s)
        
        if m < 0:
            prediction.append(-1)
        else:
            prediction.append(1)
            
    return prediction

def error(prediction, y):
    print len(prediction)
    print "y",y 
    print "pred", prediction
    print prediction
    return (prediction != y)
    


In [102]:
data, y = gen_arti()
data = np.asarray(data)
print data
N = data.shape[0]
K = np.asmatrix(np.zeros((N,N)))
lamda=1
for i,x in enumerate(data):
    for j,e in enumerate(data):
        K[i,j] = f_K(x,e)

y = y.reshape((y.shape[0], 1))
S, a = ivm(data, y)
print "S final"
for el in S:
    print el
print "Il y a ", len(S), " import points"

print "a final"
for e in a:
    print e

prediction = predict(data, S, a)
print prediction
print y
erreur = error(prediction, y)
print "Erreur de classification", erreur

[[ 0.83963201  1.13587076]
 [ 0.74486587  1.03682817]
 [ 0.95422232  0.94140071]
 [-1.13978503 -1.08703366]
 [ 0.95044575  1.04882175]
 [-0.80439897 -0.96173919]
 [-1.14909573 -0.89532812]
 [ 0.95368522  1.03952996]
 [-1.00687017 -0.94280023]
 [-0.99845549 -1.0969375 ]]
[[ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]]
[[ 1.08100253]
 [ 1.04688334]
 [ 1.06701465]
 [-3.2976375 ]
 [ 1.08701408]
 [-3.23571195]
 [-3.28338106]
 [ 1.08603154]
 [-3.31829028]
 [-3.31299058]]
[[ 1.1268377 ]
 [ 1.07932752]
 [ 1.10717296]
 [-4.7960746 ]
 [ 1.13392827]
 [-4.71316158]
 [-4.77783165]
 [ 1.13257946]
 [-4.82983894]
 [-4.81959887]]
[[ 1.12603816]
 [ 1.07752528]
 [ 1.10594253]
 [-4.90988246]
 [ 1.13319292]
 [-4.82558082]
 [-4.89138314]
 [ 1.13181675]
 [-4.94474567]
 [-4.93408786]]
[[ 1.12603618]
 [ 1.07752121]
 [ 1.10593964]
 [-4.91012018]
 [ 1.13319107]
 [-4.82581572]
 [-4.89162033]
 [ 1.13181484]
 [-4.94498573]
 [-4.93432703]]
[[ 1.12603618]
 [ 1.07752121]
 [ 1.10593965]
 [-4.9101